In [ ]:
# Imports
import os
import torch
import matplotlib.pyplot as plt
from torch.nn import BCEWithLogitsLoss 

# Configurations
base_path = os.getcwd()
save_path = os.path.join(base_path, "out")

noise_size = 50
batch_size = 32
image_size = 64
image_channel_size = 4
layer_base_width = 64

initial_learning_rate = 0.0002
min_learning_rate = initial_learning_rate / 10

In [ ]:
# Load The Data
from importlib import reload
from enum import auto, Enum
from model import dataset 
from torch.utils.data import DataLoader

reload(dataset)

class Category(Enum):
    MOB = auto()
    NPC = auto()
    
category = Category.NPC
train_path = os.path.join(base_path, f"train\\{category.name}")

transformer = dataset.get_transformer(image_size, image_channel_size)
training_set = dataset.Data(train_path, category, image_channel_size, transformer)
training_loader = DataLoader(training_set, batch_size=batch_size, shuffle=True)

training_loader.show_samples(num_samples=10)

In [ ]:
from importlib import reload
import model.adversary as adversary_module
import model.generator as generator_module
import model.discriminator as discriminator_module
import model.training as training_module
import model.log as log_module

reload(log_module)
reload(adversary_module)
reload(generator_module)
reload(discriminator_module)
reload(training_module)


def get_generator():
    generator = generator_module.Generator(noise_size, image_size, image_channel_size, layer_base_width)
    generator.add_linear_layer(out_factor=8)
    generator.add_block(in_factor=8, out_factor=4)
    generator.add_block(in_factor=4, out_factor=2)
    generator.add_block(in_factor=2, out_factor=1)
    generator.add_convolutional_layer(in_size=generator.base_width, out_size=image_channel_size)
    generator.build()

    generator.set_optimizer(learning_rate=initial_learning_rate)
    generator.set_scheduler(min_lr=min_learning_rate)
    
    return generator


def get_discriminator(generator):
    discriminator = discriminator_module.Discriminator(noise_size, image_size, image_channel_size, layer_base_width)
    discriminator.add_convolutional_layer(in_size=image_channel_size, out_size=generator.base_width)
    discriminator.add_activator_layer()
    discriminator.add_block(in_factor=1, out_factor=2,)
    discriminator.add_block(in_factor=2, out_factor=4)
    discriminator.add_block(in_factor=4, out_factor=8)
    discriminator.add_mini_batch_layer(in_factor=8, out_factor=9, inter_value=10)
    discriminator.add_linear_layer(in_factor=9)
    discriminator.build()

    discriminator.set_optimizer(learning_rate=initial_learning_rate)
    discriminator.set_scheduler(min_lr=min_learning_rate)
    
    return discriminator


configuration = {
    "data": training_loader,
    
    # Image Configurations
    "batch_size": batch_size,
    "noise_size": noise_size, 
    "image_size": image_size,
    "image_channel_size": image_channel_size,
    
    # Training Configurations
    "num_epochs": 500,
    "initial_learning_rate": initial_learning_rate,
    "min_learning_rate": min_learning_rate,
    "label_smoothing_value": 0.9,
    "gradient_clip_value": 1.0,
    "use_mixed_precision": True,
    "loss_function": BCEWithLogitsLoss(),
    "generator_train_frequency": 1,
    "discriminator_train_frequency": 1,
    "max_train_frequency": 3,
    
    # Sampling Configurations
    "sample_size": 10,
    
    # Model Saving Configurations
    "out_path": "out",
    "use_logging": True,
    "save_model_interval": 0,
    "save_sample_interval": 20,
    "save_loss_plot": True,
    "save_configuration": True,
    "delete_previous_saves": False,
}

# print(generator, "\n")
# print(discriminator)

for noise_size in [50]:
    configuration["noise_size"] = noise_size
    generator = get_generator()
    discriminator = get_discriminator(generator)
    trainer = training_module.Trainer(training_loader, generator, discriminator, configuration)
    trainer.run()